In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sproc,sum
import json
#import snowflake.snowpark

def initiateSession():

    with open('creds.json') as f:
        connection_parameters = json.load(f)

    session = Session.builder.configs(connection_parameters).create()
    return session

session = initiateSession()

In [5]:
df_orders = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS")
df_customers = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.CUSTOMER")

In [6]:
df_orders.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"O_ORDERKEY"  |"O_CUSTKEY"  |"O_ORDERSTATUS"  |"O_TOTALPRICE"  |"O_ORDERDATE"  |"O_ORDERPRIORITY"  |"O_CLERK"        |"O_SHIPPRIORITY"  |"O_COMMENT"                                         |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|16200001      |103552       |F                |98686.85        |1993-09-08     |5-LOW              |Clerk#000002872  |0                 | furiously. furiously final ideas nag carefully...  |
|16200002      |397082       |O                |304962.83       |1997-11-14     |3-MEDIUM           |Clerk#000007665  |0                 |lets. blithely brave asymptotes wake blithely u...  |
|16200003      |253015       |P         

### Join the tables on O_CUSTKEY and select few columns.

In [7]:
joined_df1 = df_orders.join(df_customers, df_orders["O_CUSTKEY"] == df_customers["C_CUSTKEY"]) \
.select(df_orders["O_ORDERKEY"], df_orders["O_TOTALPRICE"], df_customers["C_NAME"], df_customers["C_ACCTBAL"])

In [9]:
from snowflake.snowpark.functions import col, sum, avg, count, max

Apply .filter method to get the customers and orders where price > 5000 and the order status ="F"

In [10]:
df_filtered_orders = joined_df1.filter((col("O_TOTALPRICE") > 50000) & (col("O_ORDERSTATUS") == 'F'))

Apply some aggregation functions to fetch the total orders, total spend, average order price and maximum order price for each customer.

In [11]:
df_customer_aggregates = df_filtered_orders.group_by("C_NAME") \
    .agg(
    count(col("O_ORDERKEY")).alias("TOTAL_ORDERS"),
    sum(col("O_TOTALPRICE")).alias("TOTAL_SPENT"),
    avg(col("O_TOTALPRICE")).alias("AVG_ORDER_PRICE"),
    max(col("O_TOTALPRICE")).alias("MAX_ORDER_PRICE")
)

Write the final dataframe back to Snowflake.

In [12]:
df_customer_aggregates.write.mode("overwrite").save_as_table("customer_order_summary")